# Denoising con filtro de Wiener


## Contexto

Asumamos que la señal de entrada corresponde a una señal deseada (información) que ha sido contaminada con ruido aditivo

$$
u_n = d_n + \nu_n,
$$

adicionalmente asumimos que

- el ruido es estacionario en el sentido amplio y de media cero $\mathbb{E}[\nu_n] = 0$
- el ruido es blanco, es decir no tiene correlación consigo mismo o con la señal deseada

$$
r_{\nu d}(k) = 0, \forall k
$$

- el ruido tiene una cierta varianza $\mathbb{E}[\nu_n^2] = \sigma_\nu^2, \forall n$

## Filtro de Wiener

El filtro de Wiener se define como

$$
\textbf{h}^{*} = R_{uu} ^{-1} R_{ud}
$$

donde $R_{uu}$ es la matriz de autocorrelación de $u$ y $R_{ud}$ es la correalación entre $u$ y $d$

Bajo los supuestos anteriores tenemos que

- $R_{uu} = R_{dd} + R_{\nu\nu}$
- $R_{ud} = R_{dd}$

Luego el filtro óptimo será

$$
\vec h^{*} = \frac{R_{dd}}{R_{dd} + R_{\nu\nu}}
$$

y su respuesta en frecuencia

$$
H(f) = \frac{S_{dd}(f)}{S_{dd}(f) + S_{\nu\nu}(f)} = \frac{1}{1 + S_{\nu\nu}(f)/S_{dd}(f)}
$$

donde $S_{dd}$ es la densidad espectral de potencia de la señal y $S_{\nu\nu}(f)$ es la densidad espectral de potencia del ruido. La densidad espectral de potencia es la transformada de Fourier de la autocorrelación. Si el ruido es blanco entonces $S_{\nu \nu}(f) = \sigma^2$

Notemos que
- en frecuencias donde la $S_{dd}(f) > S_{\nu\nu}(f)$, entonces $H(f) = 1$
- en frecuencias donde la $S_{dd}(f) < S_{\nu\nu}(f)$, entonces $H(f) = 0$

En la práctica no conocemos la densidad espectral de $d$ pero podemos hacer supuestos sobre la razon señal a ruido $ \frac{S_{dd}(f)}{S_{\nu\nu}(f)}$

## Ejemplo

Sea la siguiente señal contaminada con ruido blanco 

Intenremos recuperar $d$ a partir de $u$ usando un filtro de wiener

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as sfft

Fs = 20
noise_std = 2
t = np.arange(0, 100, 1/Fs)
d = np.exp(-0.5 * ((t - 20.) / 5.0) ** 2)
#d = 0.2*np.cos(2.0*np.pi*t)
d = d - np.mean(d)
np.random.seed(12345)
nu =  np.random.randn(*d.shape)*noise_std
u = d + nu

freq = sfft.rfftfreq(n=len(u), d=1/Fs)
U = sfft.rfft(u)
Suu = np.real(np.multiply(U, np.conj(U)))/len(u)
D = sfft.rfft(d)
Sdd = np.real(np.multiply(D, np.conj(D)))/len(u)

fig, ax = plt.subplots(2, 1, figsize=(6, 4), tight_layout=True)
ax[0].scatter(t, u, c='k', s=2, alpha=0.75)
ax[0].plot(t, d, lw=2)
ax[0].set_xlabel('Tiempo')
ax[1].plot(freq, Suu, c='k', alpha=0.75)
ax[1].axhline(noise_std**2, c='r', ls='--', lw=2)
ax[1].plot(freq, Sdd, lw=2)
ax[1].set_ylabel('PSD');
ax[1].set_xlabel('Frecuencia [Hz]');

La función [`scipy.signal.wiener`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.wiener.html) implementa el filtro de Wiener  que actua como filtro pasa-bajo

Sus argumentos son el orden del filtro y un estimado para la varianza del ruido



In [ ]:
import scipy.signal
from ipywidgets import interact, IntSlider, FloatSlider

fig, ax = plt.subplots(2, 1, figsize=(6, 4), tight_layout=True)

def update(L, noise_var):
    for ax_ in ax:
        ax_.cla()
    y = scipy.signal.wiener(u, mysize=L+1, noise=noise_var)
    freq = sfft.rfftfreq(n=len(y), d=1/Fs)
    Y = sfft.rfft(y)
    ax[0].plot(t, d, lw=4, alpha=0.75)
    ax[0].plot(t, y, lw=2, alpha=0.75)
    ax[0].set_xlabel('Tiempo')
    ax[1].plot(freq, np.abs(D), lw=4, alpha=0.75)
    ax[1].plot(freq, np.abs(Y), lw=2, alpha=0.75)
    ax[1].set_xlabel('Frecuencia [Hz]');

interact(update, 
         L=IntSlider(min=1, max=200, step=1),
         noise_var=FloatSlider(min=0, max=6, step=0.05));    